# Resolución de Hitori mediante búsqueda informada

Para la resolución de este problema, lo haremos mediante técnicas de búsqueda en espacios de estado. Para ello debemos tener en cuenta cuál será la situación inicial desde la que se parte, el objetivo final, las diferentes situaciones por las que puede pasar y qué acciones hay que realizar para pasar de una situación a otra.

En primer lugar, vamos a necesitar una serie de importaciones para usar, entre ellas como es obvio, las importaciones de las librerías <b>problema_espacio_estados.py</b> y <b>búsqueda_espacio_estados.py</b>. Respecto a las otras dos importaciones:
- <b>Copy</b> sirve para realizar copias de objetos de manera que no sea modificable el original. Nos servirá para realizar comprobaciones sobre estados.
- <b>Label</b> sirve para obtener una tupla sobre el número de elementos conexas en una matriz y dicha matriz con los elementos sustituidos con una enumeración para cada componente conexa. Nos servirá para realizar la comprobación de una de las restricciones.

In [1]:
import problema_espacio_estados as probee
import búsqueda_espacio_estados as búsqee
import copy
from scipy.ndimage import label

Para poder realizar diferentes operaciones sobre la cuadrícula del pasatiempos Hitori, vamos a crear una clase llamada <b>Puzle</b> que nos permita realizarlas. Dicha clase tendrá como constructor las creación del objeto Puzle a partir de un array bidimensional (una matriz). A continuación se explica cada una de las operaciones:

En primer lugar, operaciones básicas:

- <b>numeroColumnas:</b> esta función devuelve el número de columnas que hay en la cuadrícula.
- <b>numeroFilas:</b> esta función devuelve el número de filas que hay en la cuadrícula.
- <b>casilla:</b> esta función devuelve el valor de una casilla concreta a partir de la fila y columna pasadas como parámetros.
- <b>casillaDentro:</b> esta función devuelve un valor booleano dependiendo de si una casilla concreta está dentro de la cuadrícula a partir de la fila y columna pasadas como parámetros.
- <b>casillaMarcada:</b> esta función devuelve un valor booleano dependiendo de si una casilla concreta está marcada en negro (se interpreta como que tiene el valor 0) a partir de la fila y columna pasadas como parámetros.
- <b>transponer:</b> esta función devuelve la cuadricula transpuesta (se cambian las filas por columnas y viceversa).
- <b>obtenerFila:</b> esta función devuelve una lista correspondiente a una fila concreta de la cuadrícula a partir de la fila pasada como parámetro.
- <b>obtenerColumna:</b> esta función devuelve una lista correspondiente a una columna concreta de la cuadrícula a partir de la columna pasada como parámetro. Para ello se utiliza la función 'transponer'.
- <b>estaRepetido:</b> esta función devuelve un valor booleano dependiendo de si un elemento se repite a lo largo de la fila y/o columna, a partir de la fila y columnas pasadas como parámetros. Gracias a esta función se podrá descartar comprobar aquellas casillas que no estén repetidas en la misma fila y/o columna.

A continuación, operaciones necesarias para realizar las acciones del problema:

- <b>marcarCasilla:</b> esta función marca en negro una casilla a partir de la fila y columna pasadas como parámetros, además como consecuencia de marcar en negro una casilla, entonces las casillas contiguas verticales y horizontales no podrán ser marcadas nunca, por lo que se usa la función 'marcarCasillasFilasColumnasContigua' que permite marcar en negro aquellas casillas repetidas en filas y columnas a esas casillas que no pueden ser marcadas nunca. Además, la función 'marcarCasilla' devolverá una lista de todas las casillas marcadas en la llamada de la función 'marcarCasillasFilasColumnasContigua', para realizar más adelante una comprobación de que no se ha incumplido alguna restricción. Gracias a esta función se podrá realizar la acción marcar múltiples casillas como consecuencia de marcar una sola casilla.
- <b>marcarCasillasFilasColumnasContigua:</b> esta función marca en negro aquellas casillas que se encuentran en la misma fila y columna que la contigua vertical y horizontal de la casilla correspondiente a la fila y columna pasada por parámetros. Dada las posibles situaciones, se realiza las comprobaciones para los 4 lados posibles (izquierda, derecha, arriba y debajo). Para cada lado se verifica que la casilla está dentro de la cuadrícula y si es así, se comprueba si tiene elementos repetidos, en tal caso, se obtienen los índices de los elementos repetidos y se marcan dichas casillas (descartando para que no se marque la casilla contigua a la correspondiente pasada por parámetros). Además, la función 'marcarCasillasFilasColumnasContigua' devolverá una lista de todas las casillas marcadas, para realizar más adelante una comprobación de que no se ha incumplido alguna restricción.

Finalmente, operaciones más complejas para realizar antes de empezar la búsqueda que nos permitirá descartar posibilidades para comprobar:

- <b>parRepetidoConsecutivo:</b> esta función comprueba si una casilla concreta dada a partir de la fila y columna pasada por parámetros, tiene a uno de sus lados un valor, y el par que formaría está de nuevo consecutivo a este (horizontalmente o verticalmente), formando un doble par igual (por ejemplo, de manera horizontal: 2 5 2 5), y por tanto, se sabrá que las dos casillas centrales no podrán marcarse en negro y como consecuencia de ello se marcarán en negros todas las que se repitan a esos dos valores para esa fila y/o columna (excluyendo como es obvio de marcar en negro esas dos casillas centrales). El marcado se realizará llamando a la función 'marcarCasilla' que como consecuencia llamará también a 'marcarCasillasFilasColumnasContigua', descartando así más posibilidades desde un inicio.
- <b>estaEntreDosRepetidos:</b> esta función comprueba si una casilla concreta dada a partir de la fila y columna pasada por parámetros, está entre dos casillas cuyo valor es el mismo, tanto de manera vertical u horizontal (por ejemplo, de manera horizontal: 2 5 2), y por tanto como una de las casillas cuyo valor es el mismo deberá marcarse, se sabe que la que está entre medio no podrá nunca marcarse. Como consecuencia de ello, se marcarán en negro todas las casillas cuyo valor sea el mismo que el valor central, tanto para su fila como su columna. El marcado se realizará llamando a la función 'marcarCasilla' que como consecuencia llamará también a 'marcarCasillasFilasColumnasContigua', descartando así más posibilidades desde un inicio. Esta función también queda aplicable a una situación en la que tuvieramos el mismo número consecutivo (por ejemplo, en horizontal: 2 2 2), esta función haría que quedaran marcados en negro los elementos de cada lado. Es importante que esta función se use después de usar la función 'parRepetidoConsecutivo', ya que si no descartaría toda posibilidad de encontrar pares repetidos con la función 'parRepetidoConsecutivo'.
- <b>parIgual:</b> esta función comprueba si una casilla concreta dada a partir de la fila y columna pasada por parámetros,  tiene a uno de sus lados (horizontalmente o verticalmente) dos casillas cuyo par tienen el mismo valor, y por tanto, se sabrá que al menos una de esas dos tendrá que ser marcada en negro durante la búsqueda y como consecuencia de ello, entonces se marcarán en negro todas las casillas que tengan el mismo valor para esa fila (si el par está horizontalmente) o esa columna (si el par está verticalmente), excluyendo a marcar las casillas que forman el par. El marcado se realizará llamando a la función 'marcarCasilla' que como consecuencia llamará también a 'marcarCasillasFilasColumnasContigua', descartando así más posibilidades desde un inicio.
- <b>repetidoContiguoEsquina:</b> esta función comprueba si las casillas de las esquinas de la cuadrícula (de manera independiente) tienen el mismo valor que sus casillas contiguas tanto horizontalmente como verticalmente una casilla (por ejemplo, para la esquina superior izquierda: 22 y debajo del primer 2, otro 2), y por tanto, se sabrá que la casilla de la esquina deberá ser marcada en negro. El marcado se realizará llamando a la función 'marcarCasilla' que como consecuencia llamará también a 'marcarCasillasFilasColumnasContigua', descartando así más posibilidades desde un inicio.
- <b>parRepetidoEsquina:</b> esta función comprueba si las casillas de las esquinas de la cuadrícula (de manera independiente) tienen dos pares, en donde cada par está compuesto del mismo número, (por ejemplo, para la esquina superior izquierda de manera horizontal: 22 y debajo 55), y por tanto, se sabrá que la casilla de la esquina deberá ser marcada en negro y la que hace esquina con esta también será marcada en negro. El marcado se realizará llamando a la función 'marcarCasilla' que como consecuencia llamará también a 'marcarCasillasFilasColumnasContigua', descartando así más posibilidades desde un inicio.


Para la <b>representación de la cuadrícula</b> como objeto Puzle se ha realizado una mejora para que sea lo más visual posible, de manera que los corchetes de la matriz y las comas son sutituidas por parades en código ASCII y adicionalmente se han añadido más paredes para los lados extremos de la cuadricula, además de meter una pared intermedia entre cada fila. Por último se han sustituido las casillas con valor a 0 por el carácter cuadrado en ASCII.

In [2]:
class Puzle:
    def __init__(self, cuadricula):
        self.cuadricula = cuadricula
    
    def numeroColumnas(self):
        return len(self.cuadricula[0])
    
    def numeroFilas(self):
        return len(self.cuadricula)
    
    def casilla(self, f, c):
        return self.cuadricula[f][c]
    
    def casillaDentro(self, f, c):
        return 0 <= f < self.numeroFilas() and 0 <= c < self.numeroColumnas()
    
    def casillaMarcada(self, f, c):
        return self.casilla(f, c) == 0
    
    def transponer(self):
        puzle = [[f[i] for f in self.cuadricula] for i in range(len(self.cuadricula[0]))]
        return puzle
    
    def obtenerFila(self, f):
        return self.cuadricula[f]
    
    def obtenerColumna(self, c):
        puzleTranspuesto = self.transponer()
        return puzleTranspuesto[c]
    
    def estaRepetido(self, f, c):
        repetido = False
        numeroCasilla = self.casilla(f, c)
        lista = self.obtenerFila(f)
        if lista.count(numeroCasilla) > 1:
            repetido = True
        if not repetido:
            lista = self.obtenerColumna(c)
            if lista.count(numeroCasilla) > 1:
                repetido = True
        return repetido
    
    def marcarCasilla(self, f, c):
        casillasMarcadas = []
        self.cuadricula[f][c] = 0
        casillasMarcadas = self.marcarCasillasFilasColumnasContigua(f, c)
        return casillasMarcadas  
    
    def marcarCasillasFilasColumnasContigua(self, f, c):
        casillasMarcadas = []
        #Contigua izquierda:
        if self.casillaDentro(f, c - 1):
            contiguaIzquierda = self.casilla(f, c - 1)
            fila = self.obtenerFila(f)
            columna = self.obtenerColumna(c - 1)
            if fila.count(contiguaIzquierda) > 1:
                indices = []
                for index, num in enumerate(fila):
                    if num == contiguaIzquierda:
                        indices.append(index)
                indices.remove(c - 1)
                for i in indices:
                    self.cuadricula[f][i] = 0
                    casillasMarcadas.append([f,i])
            if columna.count(contiguaIzquierda) > 1:
                indices = []
                for index, num in enumerate(columna):
                    if num == contiguaIzquierda:
                        indices.append(index)
                indices.remove(f)
                for i in indices:
                    self.cuadricula[i][c - 1] = 0
                    casillasMarcadas.append([i, c - 1])
        #Contigua derecha:
        if self.casillaDentro(f, c + 1):
            contiguaDerecha = self.casilla(f, c + 1)
            fila = self.obtenerFila(f)
            columna = self.obtenerColumna(c + 1)
            if fila.count(contiguaDerecha) > 1:
                indices = []
                for index, num in enumerate(fila):
                    if num == contiguaDerecha:
                        indices.append(index)
                indices.remove(c + 1)
                for i in indices:
                    self.cuadricula[f][i] = 0
                    casillasMarcadas.append([f,i])
            if columna.count(contiguaDerecha) > 1:
                indices = []
                for index, num in enumerate(columna):
                    if num == contiguaDerecha:
                        indices.append(index)
                indices.remove(f)
                for i in indices:
                    self.cuadricula[i][c + 1] = 0
                    casillasMarcadas.append([i, c + 1])
        #Contigua arriba:
        if self.casillaDentro(f - 1, c):
            contiguaArriba = self.casilla(f - 1, c)
            fila = self.obtenerFila(f - 1)
            columna = self.obtenerColumna(c)
            if fila.count(contiguaArriba) > 1:
                indices = []
                for index, num in enumerate(fila):
                    if num == contiguaArriba:
                        indices.append(index)
                indices.remove(c)
                for i in indices:
                    self.cuadricula[f - 1][i] = 0
                    casillasMarcadas.append([f - 1,i])
            if columna.count(contiguaArriba) > 1:
                indices = []
                for index, num in enumerate(columna):
                    if num == contiguaArriba:
                        indices.append(index)
                indices.remove(f - 1)
                for i in indices:
                    self.cuadricula[i][c] = 0
                    casillasMarcadas.append([i, c])
        #Contigua abajo:
        if self.casillaDentro(f + 1, c):
            contiguaAbajo = self.casilla(f + 1, c)
            fila = self.obtenerFila(f + 1)
            columna = self.obtenerColumna(c)
            if fila.count(contiguaAbajo) > 1:
                indices = []
                for index, num in enumerate(fila):
                    if num == contiguaAbajo:
                        indices.append(index)
                indices.remove(c)
                for i in indices:
                    self.cuadricula[f + 1][i] = 0
                    casillasMarcadas.append([f + 1,i])
            if columna.count(contiguaAbajo) > 1:
                indices = []
                for index, num in enumerate(columna):
                    if num == contiguaAbajo:
                        indices.append(index)
                indices.remove(f + 1)
                for i in indices:
                    self.cuadricula[i][c] = 0
                    casillasMarcadas.append([i, c])
        return casillasMarcadas                   
    
    def parRepetidoConsecutivo(self, f, c):
        #Por la izquierda
        if self.casillaDentro(f, c - 1) and self.casillaDentro(f, c - 2) and self.casillaDentro(f, c - 3):
            parRepetidoConsecutivoIzquierda = self.casilla(f, c - 1) == self.casilla(f, c - 3) and self.casilla(f, c) == self.casilla(f, c - 2)
            fila = self.obtenerFila(f)
            if parRepetidoConsecutivoIzquierda:
                indices = []
                for index, num in enumerate(fila):
                    if num == self.casilla(f, c) or num == self.casilla(f, c - 1):
                        indices.append(index)
                indices.remove(c - 1)
                indices.remove(c - 2) 
                for i in indices:
                    self.marcarCasilla(f, i)
        #Por la derecha
        if self.casillaDentro(f, c + 1) and self.casillaDentro(f, c + 2) and self.casillaDentro(f, c + 3):
            parRepetidoConsecutivoDerecha = self.casilla(f, c + 1) == self.casilla(f, c + 3) and self.casilla(f, c) == self.casilla(f, c + 2)
            fila = self.obtenerFila(f)
            if parRepetidoConsecutivoDerecha:
                indices = []
                for index, num in enumerate(fila):
                    if num == self.casilla(f, c) or num == self.casilla(f, c + 1):
                        indices.append(index)
                indices.remove(c + 1)
                indices.remove(c + 2) 
                for i in indices:
                    self.marcarCasilla(f, i)
        #Por arriba
        if self.casillaDentro(f - 1, c) and self.casillaDentro(f - 2, c) and self.casillaDentro(f - 3, c):
            parRepetidoConsecutivoArriba = self.casilla(f - 1, c) == self.casilla(f - 3, c) and self.casilla(f, c) == self.casilla(f - 2, c)
            columna = self.obtenerColumna(c)
            if parRepetidoConsecutivoArriba:
                indices = []
                for index, num in enumerate(columna):
                    if num == self.casilla(f, c) or num == self.casilla(f - 1, c):
                        indices.append(index)
                indices.remove(f - 1)
                indices.remove(f - 2) 
                for i in indices:
                    self.marcarCasilla(i, c)
        #Por abajo
        if self.casillaDentro(f + 1, c) and self.casillaDentro(f + 2, c) and self.casillaDentro(f + 3, c):
            parRepetidoConsecutivoAbajo = self.casilla(f + 1, c) == self.casilla(f + 3, c) and self.casilla(f, c) == self.casilla(f + 2, c)
            columna = self.obtenerColumna(c)
            if parRepetidoConsecutivoAbajo:
                indices = []
                for index, num in enumerate(columna):
                    if num == self.casilla(f, c) or num == self.casilla(f + 1, c):
                        indices.append(index)
                indices.remove(f + 1)
                indices.remove(f + 2) 
                for i in indices:
                    self.marcarCasilla(i, c)
    
    def estaEntreDosRepetidos(self, f, c):
        #Horizontalmente
        if self.casillaDentro(f, c - 1) and self.casillaDentro(f, c + 1):
            if self.casilla(f, c - 1) == self.casilla(f, c + 1):
                fila = self.obtenerFila(f)
                columna = self.obtenerColumna(c)
                indicesFila = []
                indicesColumna = []
                for index, num in enumerate(fila):
                    if num == self.casilla(f, c):
                        indicesFila.append(index)
                indicesFila.remove(c)
                for i in indicesFila:
                    self.marcarCasilla(f, i)
                for index, num in enumerate(columna):
                    if num == self.casilla(f, c):
                        indicesColumna.append(index)
                indicesColumna.remove(f)
                for i in indicesColumna:
                    self.marcarCasilla(i, c)
        #Verticalmente
        if self.casillaDentro(f - 1, c) and self.casillaDentro(f + 1, c):
            if self.casilla(f - 1, c) == self.casilla(f + 1, c):
                fila = self.obtenerFila(f)
                columna = self.obtenerColumna(c)
                indicesFila = []
                indicesColumna = []
                for index, num in enumerate(fila):
                    if num == self.casilla(f, c):
                        indicesFila.append(index)
                indicesFila.remove(c)
                for i in indicesFila:
                    self.marcarCasilla(f, i)
                for index, num in enumerate(columna):
                    if num == self.casilla(f, c):
                        indicesColumna.append(index)
                indicesColumna.remove(f)
                for i in indicesColumna:
                    self.marcarCasilla(i, c)
        
    def parIgual(self, f, c):
        #Por la izquierda
        if self.casillaDentro(f, c - 1) and self.casillaDentro(f, c - 2):
            esParIgualIzquierda = self.casilla(f, c - 1) == self.casilla(f, c - 2)
            fila = self.obtenerFila(f)
            if esParIgualIzquierda and fila.count(self.casilla(f, c - 1)) > 2: #Comprobamos que hay mas casillas iguales en la fila
                indices = []
                for index, num in enumerate(fila):
                    if num == self.casilla(f, c - 1):
                        indices.append(index)
                indices.remove(c - 1)
                indices.remove(c - 2) 
                for i in indices:
                    self.marcarCasilla(f, i)
        #Por la derecha
        if self.casillaDentro(f, c + 1) and self.casillaDentro(f, c + 2):
            esParIgualDerecha = self.casilla(f, c + 1) == self.casilla(f, c + 2)
            fila = self.obtenerFila(f)
            if esParIgualDerecha and fila.count(self.casilla(f, c + 1)) > 2: #Comprobamos que hay mas casillas iguales en la fila
                indices = []
                for index, num in enumerate(fila):
                    if num == self.casilla(f, c + 1):
                        indices.append(index)
                indices.remove(c + 1)
                indices.remove(c + 2) 
                for i in indices:
                    self.marcarCasilla(f, i)
        #Por la arriba
        if self.casillaDentro(f - 1, c) and self.casillaDentro(f - 2, c):
            esParIgualArriba = self.casilla(f - 1, c) == self.casilla(f - 2, c)
            columna = self.obtenerColumna(c)
            if esParIgualArriba and columna.count(self.casilla(f - 1, c)) > 2: #Comprobamos que hay mas casillas iguales en la columna
                indices = []
                for index, num in enumerate(columna):
                    if num == self.casilla(f - 1, c):
                        indices.append(index)
                indices.remove(f - 1) 
                indices.remove(f - 2) 
                for i in indices:
                    self.marcarCasilla(i, c)
       #Por la abajo
        if self.casillaDentro(f + 1, c) and self.casillaDentro(f + 2, c):
            esParIgualAbajo = self.casilla(f + 1, c) == self.casilla(f + 2, c)
            columna = self.obtenerColumna(c)
            if esParIgualAbajo and columna.count(self.casilla(f + 1, c)) > 2: #Comprobamos que hay mas casillas iguales en la columna
                indices = []
                for index, num in enumerate(columna):
                    if num == self.casilla(f + 1, c):
                        indices.append(index)
                indices.remove(f + 1)
                indices.remove(f + 2) 
                for i in indices:
                    self.marcarCasilla(i, c)    
    
    def repetidoContiguoEsquina(self):
        #Para la esquina superior izquierda
        if self.casilla(0, 0) == self.casilla(0, 1) == self.casilla(1, 0):
             self.marcarCasilla(0, 0)
        #Para la esquina superior derecha        
        if self.casilla(0, self.numeroColumnas() - 1) == self.casilla(0, self.numeroColumnas() - 2) == self.casilla(1, self.numeroColumnas() - 1):
             self.marcarCasilla(0, self.numeroColumnas() - 1)    
        #Para la esquina inferior izquierda        
        if self.casilla(self.numeroFilas() - 1, 0) == self.casilla(self.numeroFilas() - 1, 1) == self.casilla(self.numeroFilas() - 2, 0):
             self.marcarCasilla(self.numeroFilas() - 1, 0)
        #Para la esquina inferior derecha        
        if self.casilla(self.numeroFilas() - 1, self.numeroColumnas() - 1) == self.casilla(self.numeroFilas() - 1, self.numeroColumnas() - 2) == self.casilla(self.numeroFilas() - 2, self.numeroColumnas() - 1):
             self.marcarCasilla(self.numeroFilas() - 1, self.numeroColumnas() - 1)
                    
    def parRepetidoEsquina(self):
        #Para la esquina superior izquierda
        parRepetidoVerticalmente = self.casilla(0, 0) == self.casilla(1, 0) and self.casilla(0, 1) == self.casilla(1, 1)
        parRepetidoHorizontalmente = self.casilla(0, 0) == self.casilla(0, 1) and self.casilla(1, 0) == self.casilla(1, 1)
        if parRepetidoVerticalmente or parRepetidoHorizontalmente:
            self.marcarCasilla(0, 0)
            self.marcarCasilla(1, 1)
        #Para la esquina superior derecha
        parRepetidoVerticalmente = self.casilla(0, self.numeroColumnas() - 1) == self.casilla(1, self.numeroColumnas() - 1) and self.casilla(0, self.numeroColumnas() - 2) == self.casilla(1, self.numeroColumnas() - 2)
        parRepetidoHorizontalmente = self.casilla(0, self.numeroColumnas() - 1) == self.casilla(0, self.numeroColumnas() - 2) and self.casilla(1, self.numeroColumnas() - 1) == self.casilla(1, self.numeroColumnas() - 2)
        if parRepetidoVerticalmente or parRepetidoHorizontalmente:
            self.marcarCasilla(0, self.numeroColumnas() - 1)
            self.marcarCasilla(1, self.numeroColumnas() - 2)
        #Para la esquina inferior izquierda
        parRepetidoVerticalmente = self.casilla(self.numeroFilas() - 1, 0) == self.casilla(self.numeroFilas() - 2, 0) and self.casilla(self.numeroFilas() - 1, 1) == self.casilla(self.numeroFilas() - 2, 1)
        parRepetidoHorizontalmente = self.casilla(self.numeroFilas() - 1, 0) == self.casilla(self.numeroFilas() - 1, 1) and self.casilla(self.numeroFilas() - 2, 0) == self.casilla(self.numeroFilas() - 2, 1)
        if parRepetidoVerticalmente or parRepetidoHorizontalmente:
            self.marcarCasilla(self.numeroFilas() - 1, 0)
            self.marcarCasilla(self.numeroFilas() - 2, 1)
        #Para la esquina inferior derecha   
        parRepetidoVerticalmente = self.casilla(self.numeroFilas() - 1, self.numeroColumnas() - 1) == self.casilla(self.numeroFilas() - 2, self.numeroColumnas() - 1) and self.casilla(self.numeroFilas() - 1, self.numeroColumnas() - 2) == self.casilla(self.numeroFilas() - 2, self.numeroColumnas() - 2)
        parRepetidoHorizontalmente = self.casilla(self.numeroFilas() - 1, self.numeroColumnas() - 1) == self.casilla(self.numeroFilas() - 1, self.numeroColumnas() - 2) and self.casilla(self.numeroFilas() - 2, self.numeroColumnas() - 1) == self.casilla(self.numeroFilas() - 2, self.numeroColumnas() - 2)
        if parRepetidoVerticalmente or parRepetidoHorizontalmente:
            self.marcarCasilla(self.numeroFilas() - 1, self.numeroColumnas() - 1)
            self.marcarCasilla(self.numeroFilas() - 2, self.numeroColumnas() - 2)        
                    
    def __repr__(self):
        cadena = '\n╒'
        for i in range(0, self.numeroColumnas() - 1):
            cadena = cadena + '═══╤'
        cadena = cadena + '═══╕\n'
        for i in range(0, self.numeroFilas()):
            cadena = cadena + '{0}'.format(self.cuadricula[i]) + '\n'
            if i != self.numeroFilas()-1:
                cadena = cadena + '├'
                for j in range(0, self.numeroColumnas() - 1):
                    cadena = cadena + '───┼'
                cadena = cadena + '───┤\n'
        cadena = cadena + '╘'
        for i in range(0, self.numeroColumnas() - 1):
            cadena = cadena + '═══╧'
        cadena = cadena + '═══╛\n'
        cadena = cadena.replace("[", "│ ").replace("]", " │").replace(", ", " │ ").replace("0", "■")
        return cadena

Llegados a este punto, se deduce que obviamente el <b>estado inicial</b> del problema de búsqueda en espacios de estado será el objeto Puzle creado a partir de la cuadrícula que le pasemos por parámetro.

El <b>estado final</b> no lo podemos saber a priori, pero sí podemos saber que para que sea estado final se deben cumplir las siguientes restricciones:
- No puede haber dos números iguales en una fila o columna.
- No puede haber dos casillas marcadas en negro juntas horizontal o verticalmente
- Las casillas que no están marcadas en negro deben estar todas conectadas entre sí, esto es que dada una casilla no marcada en negro podemos llegar a cualquier otra moviéndonos horizontal o verticalmente sin pasar por ninguna casilla marcada en negro.

Finalmente, las <b>acciones</b> a realizar para este problema será únicamente una: marcar una casilla en negro (que a su vez se pueden llegar a marcar otras como consecuencia del marcado de esta). Esta acción será definida como una clase y tendrá una <b>aplicabilidad</b> de manera que sólo podrá aplicarse siempre que se cumpla que:
1. La casilla a marcar está dentro de la cuadrícula.
2. La casilla no está ya marcada
3. La casilla tiene un valor que realmente está repetido.
4. La casilla no tiene ya una casilla contigua horizontalmente o verticalmente ya marcada en negro
5. La cuadrícula no se hace desconexa tras marcar la casilla.
6. Adicionalmente, debido a la consecuencia de marcado en negro de otras casillas al marcar en negro otra, se comprueba que al realizar esta acción no se ha incumplido alguna de las anteriores restricciones.

Para ello será necesario redefinir la función <b>es_aplicable</b>, de la clase Acción, de la librería importada problema_espacio_estados.py.

Para la comprobación de la restricción sobre si se vuelve o no desconexa una matriz tras el marcado en negro de una casillas, hacemos uso de la librería 'label', de scipy.ndimage. Y a su vez, para poder realizar esta comprobación debemos primero hacer una copia del objeto mediante la librería 'copy'. De esta forma no quedará editado el objeto original y en caso de incumplirse la restricción no sería necesario una "acción" de desmarcado.

Lo mismo ocurrirá para la función <b>aplicar</b>, se tendrá que realizar una copia para verificar que si se incumple no se haya editado el Puzle original y se pueda entonces realizar BackTracking sobre esta.

Para la implementación de este problema, he decidido que no habrá coste, es decir, el coste será siempre 1 (o equivalente a la profundidad en la búsqueda).

In [3]:
class MarcarCasilla(probee.Acción):
    def __init__(self, f, c):
        nombre = 'Marcar posición ({0},{1})'.format(f, c)
        super().__init__(nombre)
        self.f = f
        self.c = c
   
    def casillaContiguaMarcada(self, estado, f, c):
        contiguaMarcada = False
        #Para la casilla de la izquierda 
        if estado.casillaDentro(f, c - 1):
            contiguaMarcada = estado.casillaMarcada(f, c - 1)
        #Para la casilla de la derecha
        if estado.casillaDentro(f, c + 1) and not contiguaMarcada:
            contiguaMarcada = estado.casillaMarcada(f, c + 1)
        #Para la casilla de arriba
        if estado.casillaDentro(f - 1, c) and not contiguaMarcada:
            contiguaMarcada = estado.casillaMarcada(f - 1, c)
        #Para la casilla de abajo
        if estado.casillaDentro(f + 1, c) and not contiguaMarcada:
            contiguaMarcada = estado.casillaMarcada(f + 1, c)
        return contiguaMarcada
    
    def grupoAislado(self, estado, f, c):
        estaAislada = False
        nuevoEstado = copy.deepcopy(estado)
        nuevoEstado.marcarCasilla(f, c)
        numeroComponentesConexas = label(nuevoEstado.cuadricula)
        if numeroComponentesConexas[1] > 1:
            estaAislada = True
        return estaAislada 
    
    def malMarcadasConsecuenciaPorContiguasMarcada(self, estado, f, c):
        malMarcado = False
        nuevoEstado = copy.deepcopy(estado)
        casillasMarcadas = nuevoEstado.marcarCasilla(f, c)
        malMarcado = self.grupoAislado(nuevoEstado, f, c)
        if not malMarcado:
            for casilla in casillasMarcadas:
                if self.casillaContiguaMarcada(nuevoEstado, casilla[0], casilla[1]):
                    malMarcado = True
                    break
        return malMarcado
    
    def es_aplicable(self, estado):
        return estado.casillaDentro(self.f, self.c) and not estado.casillaMarcada(self.f, self.c) and estado.estaRepetido(self.f, self.c) and not self.casillaContiguaMarcada(estado, self.f, self.c) and not self.grupoAislado(estado, self.f, self.c) and not self.malMarcadasConsecuenciaPorContiguasMarcada(estado, self.f, self.c)
    
    def aplicar(self, estado):
        nuevoEstado = copy.deepcopy(estado)
        nuevoEstado.marcarCasilla(self.f, self.c)
        return nuevoEstado

Una vez se ha definido la clase correspondiente a la única acción que tendremos, con su correspondiente aplicabilidad, vamos a definir una clase llamada Hitori, en la cual se realizará la construcción del problema como búsqueda en espacios de estado.

La cantidad de <b>acciones posibles</b> dependerá del estado en el que se encuentre en el momento el Puzle. En un principio, para el estado inicial, la cantidad de acciones posibles será el número de casillas que hay en total, es decir, será contemplado como N x M, donde N es el número de filas del Puzle y M es el número de columnas del Puzle.

Antes de empezar la búsqueda, con el estado inicial del Puzle, podemos saber que una serie de casillas serán marcadas en negro, reduciendo así la búsqueda dependiendo de la cantidad de casillas que se hayan marcado en negro según la dimensión del Puzle. Para marcar en negro estas casillas que sabemos que serán marcadas obligatoriamente, llamamos a todas las funciones de las operaciones complejas descritas en la clase Puzle.

Finalmente, se debe redefinir la función <b>es_estado_final</b> de la clase ProblemaEspacioEstados, de la librería importada problema_espacio_estados.py, donde comprobamos para cada estado obtenido si se trata del estado final o debe continuar la búsqueda. Para que sea estado final, se deben cumplir las 3 restricciones comentadas en el anterior bloque.

In [4]:
class Hitori(probee.ProblemaEspacioEstados):
    def __init__(self, cuadricula):
        acciones = []
        estado_inicial = Puzle(cuadricula)
        n_filas = estado_inicial.numeroFilas()
        m_columnas = estado_inicial.numeroColumnas()
        print('Estado inicial:', estado_inicial)
        estado_inicial.repetidoContiguoEsquina()
        estado_inicial.parRepetidoEsquina()
        for f in range(0, n_filas):
            for c in range(0, m_columnas):
                estado_inicial.parRepetidoConsecutivo(f, c)
                estado_inicial.estaEntreDosRepetidos(f, c)
                estado_inicial.parIgual(f, c)
                acciones.append(MarcarCasilla(f, c))      
        super().__init__(acciones, estado_inicial)
        self.n = n_filas
        self.m = m_columnas
    
    def es_estado_final(self, estado):
        esFinal = True
        #No se repiten números en la misma fila
        for f in range(0, self.n):
            lista = estado.obtenerFila(f)
            for i in lista:
                if i != 0:
                    if lista.count(i) > 1:
                        esFinal = False
                        return esFinal      
        #No se repiten números en la misma columna
        for c in range(0, self.m):
            lista = estado.obtenerColumna(c)
            for i in lista:
                if i != 0:
                    if lista.count(i) > 1:
                        esFinal = False
                        return esFinal
        #No hay 2 casillas contiguas marcadas en negro
        for f in range(0, self.n):
            for c in range(0, self.m):
                if estado.casillaDentro(f, c - 1):
                    if estado.casillaMarcada(f, c - 1) and estado.casillaMarcada(f, c):
                        esFinal = False
                        return esFinal
                if estado.casillaDentro(f, c + 1):
                    if estado.casillaMarcada(f, c + 1) and estado.casillaMarcada(f, c):
                        esFinal = False
                        return esFinal
                if estado.casillaDentro(f - 1, c):
                    if estado.casillaMarcada(f - 1, c) and estado.casillaMarcada(f, c):
                        esFinal = False
                        return esFinal
                if estado.casillaDentro(f + 1, c):
                    if estado.casillaMarcada(f + 1, c) and estado.casillaMarcada(f, c):
                        esFinal = False
                        return esFinal                 
        #No se ha hecho desconexa la matriz
        numeroComponentesConexas = label(estado.cuadricula)
        if numeroComponentesConexas[1] > 1:
            esFinal = False
            return esFinal
        print('Estado final (solución):', estado)
        return esFinal

Para la búsqueda A* será necesario definir una heurística. He tomado la decisión de que una buena heurística sería comprobar si se repiten elementos en una fila y/o columna, de manera que el valor de la heurística varia tal que así:
- Si se repiten elementos en alguna fila y columna, el valor de la heurística será 2.
- Si se repiten elementos en alguna fila o columna, el valor de la heurística será 1
- Si no se repiten elementos en ninguna fila y columna (es decir, posiblemente sea estado final), el valor de la heurística será 0.

In [5]:
def heuristica(nodo):
    estado = nodo.estado
    sumaCantidadNumerosRepetidos = 0
    for f in range(0, estado.numeroFilas()):
        lista = estado.obtenerFila(f)
        for i in lista:
            if i != 0:
                if lista.count(i) > 1:
                    sumaCantidadNumerosRepetidos = sumaCantidadNumerosRepetidos + 1
                    break
    for c in range(0, estado.numeroColumnas()):
        lista = estado.obtenerColumna(c)
        for i in lista:
            if i != 0:
                if lista.count(i) > 1:
                    sumaCantidadNumerosRepetidos = sumaCantidadNumerosRepetidos + 1
                    break
    return sumaCantidadNumerosRepetidos

Por último, vamos a implementar una función que, a partir de una matriz de números naturales de tamaño MxN (no necesariamente cuadrada), construya el puzle hitori correspondiente a dicha matriz y lo resuelva mediante un algoritmo de búsqueda que también se pasa como argumento, para finalmente devolver la cuadrícula del puzle hitori resuelta.

In [6]:
def resuelveHitori(tipoBusqueda, cuadricula):
    aplicaciones = ''
    try:
        problemaPuzle = Hitori(cuadricula)
        diccionarioBusquedas = {
            'bAnchura': búsqee.BúsquedaEnAnchura(),
            'bProfundidad': búsqee.BúsquedaEnProfundidad(),
            'bOptima': búsqee.BúsquedaÓptima(),
            'bAEstrella': búsqee.BúsquedaAEstrella(heuristica)
        }
        busqueda = diccionarioBusquedas.get(tipoBusqueda, 'No ha especificado un tipo de búsqueda adecuado')
        busqueda.buscar(problemaPuzle)
    except AttributeError:
        print('\033[41m' + busqueda + '\033[0m')
    except:
        print('\033[41m' + '\nNo se ha encontrado ninguna solución' + '\033[0m')

Finalmente llamamos a dicha función especificando el tipo de búsqueda a usar y la cuadrícula a resolver, la cual se podrá leer de alguna línea de un fichero o definir directamente en la variable 'cuadricula'. Con la función mágica <b>timeit</b> obtendremos el promedio de tiempo que tarda en resolverse. La última cuadrícula obtenida será la solución del problema.

In [7]:
%%timeit -n1 -r1
n_linea = 20 #Indicar el número de la línea del fichero
try:
    if n_linea > 0:
        lineaFichero = [line.rstrip('\n') for line in open('ejemplos_prueba.txt')][n_linea - 1]
    #cuadricula = [[3,4,5,5,1,3],[5,6,2,3,2,1],[5,3,1,4,5,4],[1,4,3,4,2,2],[3,1,6,1,4,5],[1,2,1,5,3,4]]
    cuadricula = eval(lineaFichero.split()[0])
    resuelveHitori('bAEstrella', cuadricula)
except:
    print('\033[41m' + '\nNo hay nada para leer en esta línea o está fuera del rango del número de líneas' + '\033[0m')

Estado inicial: 
╒═══╤═══╤═══╤═══╤═══╤═══╕
│ 3 │ 4 │ 5 │ 5 │ 1 │ 3 │
├───┼───┼───┼───┼───┼───┤
│ 5 │ 6 │ 2 │ 3 │ 2 │ 1 │
├───┼───┼───┼───┼───┼───┤
│ 5 │ 3 │ 1 │ 4 │ 5 │ 4 │
├───┼───┼───┼───┼───┼───┤
│ 1 │ 4 │ 3 │ 4 │ 2 │ 2 │
├───┼───┼───┼───┼───┼───┤
│ 3 │ 1 │ 6 │ 1 │ 4 │ 5 │
├───┼───┼───┼───┼───┼───┤
│ 1 │ 2 │ 1 │ 5 │ 3 │ 4 │
╘═══╧═══╧═══╧═══╧═══╧═══╛

Estado final (solución): 
╒═══╤═══╤═══╤═══╤═══╤═══╕
│ ■ │ 4 │ 5 │ ■ │ 1 │ 3 │
├───┼───┼───┼───┼───┼───┤
│ 5 │ 6 │ 2 │ 3 │ ■ │ 1 │
├───┼───┼───┼───┼───┼───┤
│ ■ │ 3 │ 1 │ ■ │ 5 │ 4 │
├───┼───┼───┼───┼───┼───┤
│ 1 │ ■ │ 3 │ 4 │ 2 │ ■ │
├───┼───┼───┼───┼───┼───┤
│ 3 │ 1 │ 6 │ ■ │ 4 │ 5 │
├───┼───┼───┼───┼───┼───┤
│ ■ │ 2 │ ■ │ 5 │ 3 │ ■ │
╘═══╧═══╧═══╧═══╧═══╧═══╛

399 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
